In [1]:
import requests
import pandas as pd

In [2]:
def get_top_movies_per_month(year_list):
    # scapes every top movies of every month for every year in argument list
        frames = []
        
        for year in year_list:
            for month in range(1,13):
                url = 'https://www.the-numbers.com/movies/report/All/All/All/All/All/All/All/All/All/None/None/{}/{}/{}/{}/'\
                'None/None/None/None?show-release-date=On&view-order-by=domestic-box-office&view-order-direction=desc&show-'\
                'domestic-box-office=On&show-genre=On'.format(year, year, month, month)
                
                html_page = requests.get(url)
                frames.append(pd.read_html(html_page.text, index_col=None)[0])
                
        return pd.concat(frames)
    
    
def no_char(col):
    # enter dataframe column to return string of the column back w/o '$' and ','.
    df1 = col
   
    df1 = df1.str.replace('$', '')
    df1 = df1.str.replace(',', '')
    
    return df1

In [ ]:
# create a list of years to scrape
year_list = [2020, 2019, 2018, 2017, 2016]

#creates a dataframe of scraped data
df = get_top_movies_per_month(year_list)

In [ ]:
#renames column and limits data to movies that made at least a dollar
df = df.rename(columns={"DomesticBox Office":"DomesticBox_Office"})
df = df[df.DomesticBox_Office != "$0" ]

#takes the '$' and ',' out of the string. Example: "$1,200,000" to "12000000"
df["DomesticBox_Office"] = no_char(df["DomesticBox_Office"])

#because there isn't any missing values in DomesticBox_office, we can immediatly change column to int
df['DomesticBox_Office'] = df['DomesticBox_Office'].astype(int)

#filling nans in 'Genre' column to "other" 
df['Genre'] = df['Genre'].fillna("Other")

#sort rows by DomesticBox_Office values
df = df.sort_values("DomesticBox_Office", ascending=False)

#reset index based of rank
df = df.reset_index()

#convert 'Released' column to Datetime to sparse out new columns 'Month' and 'year'
df['Released'] = pd.to_datetime(df['Released'])
df['Month'] = df['Released'].dt.month
df['Year'] = df['Released'].dt.year

#dropping unnecessary columns
df = df.drop("Released", axis=1)
df = df.drop("Unnamed: 0", axis=1)
df = df.drop("index", axis=1)

In [ ]:
#exports the cleaned table to a .csv file
#also makes for a nice save point for future work
df.to_csv('Cleaned_movie_data.csv')

In [ ]:
df